## Extracting local features

In [ ]:
%%time
import os
import sys
import timm
import torch
from tqdm import tqdm
from torchinfo import summary
import numpy as np
import torchvision.transforms as ttf
import numpy as np
import csv
# !pip install nbformat
%run model_notebook.ipynb

# Setup the paths and constants
BACKBONE = "resnext"
POOL = "GeM"
NORM = None
BATCH_SIZE = 1

# Choose the image you want to use (the corresponding query file will be loaded)
# image_file_name = 'x3vA7Bk0HNI6rGkDpDZQUQ'
image_file_name = 'SQNDJeXa8UQ9pHht-13PNg'

root_dir = 'global_feature_verification_dir/'
q_idx = os.path.join(root_dir,'cph',image_file_name+'_query.json')

# Convert any given npy features file to csv file
def convert_npy_features_to_csv(source_file, target_file):
    source_features = np.load(source_file)
    source_features_list = source_features.tolist()

    with open(target_file, "w") as f:
        wr = csv.writer(f)
        wr.writerows(source_features_list)

def extract_features(net, feats_file,f_length=1024):
    # To run the feature extraction just like nicola we need the following things:
    image_size = [480,640]
    image_t = ttf.Compose([ttf.Resize(size=(image_size[0],image_size[1])),
                        ttf.ToTensor(),
                        ttf.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                        ])
    dl = create_dataloader("test",root_dir, q_idx,None,image_t, BATCH_SIZE)
    
    if not os.path.exists(feats_file):
        feats = np.zeros((len(dl.dataset), f_length))
        for i, batch in tqdm(enumerate(dl), desc="Extracting features"):
            local_features,_ = net.forward(batch.cuda())
            feats[i * dl.batch_size:i * dl.batch_size + dl.batch_size] = local_features.cpu().detach().squeeze(0)
        np.save(feats_file, feats)
        print(f"{feats_file} has been saved..........")
    else:
        print(feats_file,"already exists. Skipping.")


## Extract Local features using using the model from the disk 
model_file_weights = os.path.join('generalized_contrastive_loss','Models','MSLS','MSLS_resnext_GeM_480_GCL.pth')
model = create_model(BACKBONE,POOL,norm=None,mode="single" )
file_name_extension = ''
try:
    model.load_state_dict(torch.load(model_file_weights)["model_state_dict"])
except:
    model.load_state_dict(torch.load(model_file_weights)["state_dict"])

if torch.cuda.is_available():
    model.cuda()

# Force inference mode
model.eval()

feats_file= os.path.join(root_dir,'cph',image_file_name+'_local_feature_file'+ file_name_extension+'.npy')
csv_file= os.path.join(root_dir,'cph',image_file_name+'_local_feature_file'+file_name_extension +'.csv')

extract_features(model,feats_file)

# We are here, it means the npy is saved, convert to csv file
convert_npy_features_to_csv(feats_file, csv_file)

## Extracing local features for the whole dataset

In [ ]:
%%time
import os
import sys
import timm
import torch
from tqdm import tqdm
from torchinfo import summary
import numpy as np
import torchvision.transforms as ttf
import numpy as np
import csv
# !pip install nbformat
%run model_notebook.ipynb

In [ ]:
#  extract_features_msls(params.subset, params.root_dir, test_net, f_length, image_t, savename, results_dir, params.batch_size, 30)

# python3 extract_predictions.py --dataset MSLS --root_dir "/home/jovyan/AIR/generalized_contrastive_loss/MSLS/"
#  --subset val --model_file Models/MSLS/MSLS_resnext_GeM_480_GCL.pth --backbone resnext --pool GeM --f_length 2048 --batch_size 5

def extract_features(dl, model,f_length,  feats_file,):
    if not os.path.exists(feats_file):
        feats = np.zeros((len(dl.dataset), f_length))
        for i, batch in tqdm(enumerate(dl), desc="Extracting features"):
            local_features,_ = model.forward(batch.cuda())
            feats[i * dl.batch_size:i * dl.batch_size + dl.batch_size] = local_features.cpu().detach().squeeze(0)
        np.save(feats_file, feats)
        print(f"{feats_file} has been saved..........")
    else:
        print(feats_file,"already exists. Skipping.")


def extract_features_msls(
    model,
    subset='val',
    root_dir = 'generalized_contrastive_loss/MSLS/',
    weights_file_path = 'Models/MSLS/MSLS_resnext_GeM_480_GCL.pth',
    f_length = 1024,
    results_dir = 'generalized_contrastive_loss/results/MSLS/val/',
    batch_size = 1,
    k = 30,
    ):
    cities = ["cph", "sf"]

    savename= weights_file_path.split("/")[-1].split(".")[0]
    
    subset_dir=subset if subset == "test" else "train_val"
    image_size = [480,640]
    image_t = ttf.Compose([ttf.Resize(size=(image_size[0],image_size[1])),
                        ttf.ToTensor(),
                        ttf.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                        ])
    for c in cities:
        print(c)
        m_raw_file = root_dir+subset_dir+"/"+c+"/database/raw.csv"
        q_idx_file = root_dir+subset_dir+"/"+c+"/query.json"
        m_idx_file = root_dir+subset_dir+"/"+c+"/database.json"
        q_dl = create_dataloader("test", root_dir, q_idx_file, None, image_t, batch_size)
        q_feats_file =results_dir+"/"+savename+"_"+c+"_local_queryfeats.npy"
        
        extract_features(q_dl, model, f_length, q_feats_file)
        
        m_dl = create_dataloader("test", root_dir, m_idx_file, None, image_t, batch_size)
        m_feats_file =results_dir+"/"+savename+"_"+c+"_local_mapfeats.npy"
        
        extract_features(m_dl, model, f_length, m_feats_file)

In [ ]:
%%time
# Convert any given npy features file to csv file
def convert_npy_features_to_csv(source_file, target_file):
    source_features = np.load(source_file)
    source_features_list = source_features.tolist()

    with open(target_file, "w") as f:
        wr = csv.writer(f)
        wr.writerows(source_features_list)

# Setup the paths and constants
BACKBONE = "resnext"
POOL = "GeM"
NORM = None

## Extract Local features using using the model from the disk 
model_file_weights = os.path.join('generalized_contrastive_loss','Models','MSLS','MSLS_resnext_GeM_480_GCL.pth')
model = create_model(BACKBONE,POOL,norm=None,mode="single" )
file_name_extension = ''
try:
    model.load_state_dict(torch.load(model_file_weights)["model_state_dict"])
except:
    model.load_state_dict(torch.load(model_file_weights)["state_dict"])

if torch.cuda.is_available():
    model.cuda()

# Force inference mode
model.eval()

extract_features_msls(model)

# We are here, it means the npy is saved, convert to csv file
# convert_npy_features_to_csv(feats_file, csv_file)